In [1]:
import pandas as pd
import altair as alt
import numpy as np
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [2]:
data = pd.read_csv('Casos_Covid_2022.csv')

In [3]:
data = data[data['Fecha de diagnóstico'] >= '2022-01-01'].sort_values(by='Fecha de diagnóstico')

In [75]:
data.columns

Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
       'Código DIVIPOLA departamento', 'Nombre departamento',
       'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad',
       'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
       'Ubicación del caso', 'Estado', 'Código ISO del país',
       'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas',
       'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación',
       'Tipo de recuperación', 'Pertenencia étnica',
       'Nombre del grupo étnico'],
      dtype='object')

#### Procesamiento de datos

In [4]:
data = data[data['Recuperado'].notna()]
data['Fecha de notificación'] = pd.to_datetime(data['Fecha de notificación'])
data['Fecha de diagnóstico'] = pd.to_datetime(data['Fecha de diagnóstico'])

#### Primera Pregunta
¿Es seguro quitar las medidas de uso de tapabocas en interiores y transporte masivo?

In [109]:
covid_cases = data.groupby(by='Fecha de diagnóstico').count().rename(
    columns={'ID de caso': 'Casos'})['Casos'].reset_index()

In [110]:
covid_cases_chart = alt.Chart(covid_cases).encode(x='Fecha de diagnóstico:T', y='Casos:Q')

In [115]:
alt.layer(
    covid_cases_chart.mark_line()
).interactive()

alt.LayerChart(...)

In [116]:
covid_death_cases = data[data['Recuperado'] == 'Fallecido']
covid_death_cases_grouped = covid_death_cases.groupby(by='Fecha de diagnóstico').count().rename(
    columns={'ID de caso': 'Casos'})['Casos'].reset_index()

In [117]:
covid_cases_chart = alt.Chart(covid_death_cases_grouped).encode(x='Fecha de diagnóstico:T', y='Casos:Q')

In [119]:
alt.layer(
    covid_cases_chart.mark_line()
).interactive()

alt.LayerChart(...)

Teniendo en cuenta la cantidad de casos positivos presentados a lo largo del tiempo, se evidencia que la curva de casos ha disminuido sustancialmente, por tanto es posible que la medida restrictiva del uso del tapabocas sea innecesaria dada la cantidad de personas vacunadas, esto sumado a que se ve que la vacuna ha sido efectiva. Asi mismo, la cantidad de personas que han muerto este año por Covid es mucho menor, presentando el ultimo mes cantidades maximas de 3 personas por día. 

De cualquier forma, tener datos de las personas fallecidas podría ser de utilidad. Esto para verificar si la persona está o no vacunada.

#### Segunda pregunta

¿Existe alguna tendencia entre la cantidad de casos y el sexo?

In [133]:
gender_vs_cases = data.loc[: , ['ID de caso','Fecha de diagnóstico', 'Sexo']]

In [134]:
gender_vs_cases['Sexo'] = gender_vs_cases['Sexo'].apply(lambda x: x.upper())

In [135]:
gender_vs_cases = gender_vs_cases.groupby(by=['Fecha de diagnóstico', 'Sexo']).agg({'ID de caso': 'count'}).rename(
    columns={'ID de caso': 'Casos'}).reset_index()

In [138]:
cases = alt.Chart(gender_vs_cases).encode(x='Fecha de diagnóstico:T', y='Casos:Q', color='Sexo')

In [139]:
alt.layer(
    cases.mark_line()
).interactive()

alt.LayerChart(...)

In [155]:
pivoted_table = gender_vs_cases.pivot('Sexo', 'Fecha de diagnóstico').T

In [160]:
relative_diff = (pivoted_table['F'] - pivoted_table['M']) / pivoted_table['F']

In [163]:
relative_diff.mean() * 100

20.41867864758715

Se ve una tendencia alta al contagio mayor por parte de las mujeres con una diferencia relativa del 20%, lo que significa que de cada 10 contagios, aproximadamente 6 o 7 son mujeres

#### Tercera pregunta

¿En qué departamentos se presento un numero mayor de casos?

In [222]:
data['Nombre departamento'] = data['Nombre departamento'].str.upper()
data.loc[data['Nombre departamento'] == 'BOGOTA', 'Nombre departamento'] = 'CUNDINAMARCA'

In [223]:
department_cases = data.loc[: ,['ID de caso', 'Nombre departamento']].groupby(by=['Nombre departamento']).count().rename(
    columns={'ID de caso': 'Casos'}).reset_index()

In [224]:
top_5th_departments = department_cases.sort_values(by='Casos', ascending=False)[:5]

In [232]:
base = alt.Chart(top_5th_departments).encode(
    theta=alt.Theta("Casos:Q", stack=True), color=alt.Color("Nombre departamento:N")
)

pie = base.mark_arc(outerRadius=130)
text = base.mark_text(radius=170, size=10).encode(text="Nombre departamento:N")

pie + text

alt.LayerChart(...)

In [237]:
department_population = pd.DataFrame([
    {
        'Nombre departamento': 'ANTIOQUIA',
        'Poblacion': 6407000
    }, 
    {
        'Nombre departamento': 'BARRANQUILLA',
        'Poblacion': 1206000
    }, 
    {
        'Nombre departamento': 'CUNDINAMARCA',
        'Poblacion': 2919000
    }, 
    {
        'Nombre departamento': 'SANTANDER',
        'Poblacion': 2185000
    }, 
    {
        'Nombre departamento': 'VALLE',
        'Poblacion': 4476000
    }, 
])

In [241]:
top_5th_departments['Nombre departamento']

15    CUNDINAMARCA
1        ANTIOQUIA
32           VALLE
28       SANTANDER
4     BARRANQUILLA
Name: Nombre departamento, dtype: object

In [243]:
top_5th_departments_normalized = top_5th_departments.merge(department_population, on='Nombre departamento')

In [247]:
top_5th_departments_normalized['Casos normalizados'] = top_5th_departments_normalized['Casos'] / top_5th_departments_normalized['Poblacion']

In [248]:
base = alt.Chart(top_5th_departments_normalized).encode(
    theta=alt.Theta("Casos normalizados:Q", stack=True), color=alt.Color("Nombre departamento:N")
)

pie = base.mark_arc(outerRadius=130)
text = base.mark_text(radius=170, size=10).encode(text="Nombre departamento:N")

pie + text

alt.LayerChart(...)

Se concluye que cundinamarca es el departamento con más casos